# Rust Functions

In [2]:
fn gcd(mut n: u64, mut m: u64) -> u64 {
    assert!(n != 0 && m != 0);
    while m != 0 {
        if m < n {
            let t = m;
            m = n;
            n = t;
        }
        m = m % n;
    }
    n
}


In [4]:
// block expression
{
    print!("evaluating");
    let a = 1i32;
    a
}

1

# Writing and Running Unit Tests

In [14]:
// #[test]
fn test_gcd() {
    assert_eq!(gcd(14, 15), 1);
    assert_eq!(gcd(2 * 3 * 5 * 11 * 17, 3 * 7 * 11 * 13 * 19), 3 * 11)
}

In [15]:
test_gcd();

# Handling Command-Line Arguments

In [23]:
// https://github.com/evcxr/evcxr/blob/main/evcxr_input/src/lib.rs
:dep evcxr_input = "1.0.0"

use evcxr_input::get_string;
let a = get_string("here goes the argument: ");
a

"aaa"

In [29]:
use std::str::FromStr;
use std::env;

fn main() {
    let mut numbers = Vec::new();

    if env::args().len() > 1 {
        for arg in env::args().skip(1) {
            // Result
            numbers.push(u64::from_str(&arg).expect("error parsing argument"));
        }
    } else {
        let input: String = get_string("here goes the argument: ");
        let args: Vec<&str> = input.split(' ').collect();
        for arg in args {
            numbers.push(u64::from_str(&arg).expect("error parsing argument"));
        }
    }

    if numbers.len() == 0 {
        eprintln!("Usage: gcd NUMBER ...");
        std::process::exit(1);
    }

    let mut d = numbers[0];
    for m in &numbers[1..] {
        d = gcd(d, *m);
    }

    println!("The gcd of {:?} is {}", numbers, d);
}


In [30]:
main();

The gcd of [4, 5, 6, 7, 8] is 1


# Serving Pages to the Web

In [ ]:
// WARN: rather slow!!!
:dep actix-web = "4.9.0"
:dep serde = { version = "1.0.213", features = ["derive"] }

In [ ]:
/// Example of actix-web
/// https://actix.rs/docs/getting-started
use actix_web::{web, App, HttpResponse, HttpServer, Responder};
use serde::Deserialize;

#[derive(Deserialize)]
struct GcdParameters {
    n: u64,
    m: u64,
}

async fn get_index() -> impl Responder {
    HttpResponse::Ok()
        .content_type("text/html")
        // raw string
        .body(
            r#"
        <title>GCD</title>
        <form action="/gcd" method="post">
            <input type="text" name="n" />
            <input type="text" name="m" />
            <button type="submit">Compute</button>
        </form>
        "#,
        )
}

async fn post_gcd(form: web::Form<GcdParameters>) -> impl Responder {
    if form.m == 0 || form.m == 0 {
        return HttpResponse::BadRequest()
            .content_type("text/html")
            .body("Zero argument!");
    }

    let response = format!(
        "The GCD of {} and {} is <b>{}</b>\n",
        form.n,
        form.m,
        gcd(form.n, form.m)
    );

    HttpResponse::Ok().content_type("text/html").body(response)
}

fn gcd(mut n: u64, mut m: u64) -> u64 {
    assert!(n != 0 && m != 0);
    while m != 0 {
        if m < n {
            let t = m;
            m = n;
            n = t;
        }
        m = m % n;
    }
    n
}

#[actix_web::main]
async fn main() -> std::io::Result<()> {
    // closure expression: ||
    let server = HttpServer::new(|| {
        App::new()
            .route("/", web::get().to(get_index))
            .route("/gcd", web::post().to(post_gcd))
    });

    println!("Serving on http://localhost:3000...");
    server
        .bind("127.0.0.1:3000")
        .expect("error binding server to address")
        .run()
        .await
}

# Concurrency

In [ ]:
:dep num = "0.4.3"
:dep image = "0.25.4"
:dep crossbeam = "0.8.4"

In [ ]:
//! 输出Mandelbrot集.

use image::ColorType;
use num::Complex;

/// 将像素转换为复数.
#[allow(dead_code)]
fn pixel_to_point(
    bounds: (usize, usize),
    pixel: (usize, usize),
    upper_left: Complex<f64>,
    lower_right: Complex<f64>
) -> Complex<f64> {
    let (width, height) = (lower_right.re - upper_left.re, upper_left.im - lower_right.im);

    Complex {
        re: upper_left.re + ((pixel.0 as f64) * width) / (bounds.0 as f64),
        im: upper_left.im - ((pixel.1 as f64) * height) / (bounds.1 as f64),
    }
}

/// 在像素缓冲区中表示Mandelbrot集
#[allow(dead_code)]
pub fn render(
    pixels: &mut [u8],
    bounds: (usize, usize),
    upper_left: Complex<f64>,
    lower_right: Complex<f64>
) {
    assert_eq!(pixels.len(), bounds.0 * bounds.1);

    for row in 0..bounds.1 {
        for column in 0..bounds.0 {
            let point = pixel_to_point(bounds, (column, row), upper_left, lower_right);

            pixels[row * bounds.0 + column] = match escape_time(point, 255) {
                None => 0,
                Some(count) => 255 - (count as u8),
            };
        }
    }
}

/// 并发的`render`
#[allow(dead_code)]
pub fn render_c(
    pixels: &mut [u8],
    bounds: (usize, usize),
    upper_left: Complex<f64>,
    lower_right: Complex<f64>
) {
    let threads = 8;
    let rows_per_band = bounds.1 / threads + 1;
    {
        let bands: Vec<&mut [u8]> = pixels.chunks_mut(rows_per_band * bounds.0).collect();

        crossbeam
            ::scope(|spawner| {
                for (i, band) in bands.into_iter().enumerate() {
                    let top = rows_per_band * i;
                    let height = band.len() / bounds.0;
                    let band_bounds = (bounds.0, height);
                    let band_upper_left = pixel_to_point(bounds, (0, top), upper_left, lower_right);
                    let band_lower_right = pixel_to_point(
                        bounds,
                        (bounds.0, top + height),
                        upper_left,
                        lower_right
                    );

                    spawner.spawn(move |_| {
                        render(band, band_bounds, band_upper_left, band_lower_right);
                    });
                }
            })
            .unwrap();
    }
}

/// 确定`c`是否在Mandelbrot集中, 使用最多`limit`次迭代.
#[allow(dead_code)]
fn escape_time(c: Complex<f64>, limit: u32) -> Option<u32> {
    let mut z = Complex { re: 0.0, im: 0.0 };
    for i in 0..limit {
        z = z * z + c;
        if z.norm_sqr() > 4.0 {
            return Some(i);
        }
    }

    None
}

/// 将像素写入图片.
pub fn write_image(
    filename: &str,
    pixels: &[u8],
    bounds: (usize, usize)
) -> Result<(), std::io::Error> {
    image::save_buffer(filename, &pixels, bounds.0 as u32, bounds.1 as u32, ColorType::L8).unwrap();
    Ok(())
}

fn main() {
    use num::Complex;
    let bounds = (4000, 3000);
    let upper_left = Complex { re: -1.2, im: 0.35 };
    let lower_right = Complex { re: -1.0, im: 0.2 };
    let mut pixels = vec![0; bounds.0 * bounds.1];

    // render(&mut pixels, bounds, upper_left, lower_right);
    render_c(&mut pixels, bounds, upper_left, lower_right);

    write_image("mandelbrot.png", &pixels, bounds).expect("error writing PNG file");
}


In [ ]:
use num::Complex;
let bounds = (4000, 3000);
let upper_left = Complex {
    re: -1.2,
    im: 0.35,
};
let lower_right = Complex { re: -1.0, im: 0.2 };
let mut pixels = vec![0; bounds.0 * bounds.1];

// render(&mut pixels, bounds, upper_left, lower_right);
render_c(&mut pixels, bounds, upper_left, lower_right);

write_image("mandelbrot.png", &pixels, bounds).expect("error writing PNG file");


# Filesystems and Command-Line Tools

In [2]:
// https://github.com/evcxr/evcxr/blob/main/evcxr_input/src/lib.rs
:dep evcxr_input = "1.0.0"

In [5]:
use evcxr_input::get_string;
#[derive(Debug)]
struct Arguments {
    target: String,
    replacement: String,
    filename: String,
    output: String
}

fn parse_args() -> Arguments {
    let input = get_string("here goes the argument: ");
    let args: Vec<&str> = input.split(' ').collect();

    if args.len() != 4 {
        eprintln!("wrong number of arguments, expect [target] [replacement] [filename] [output]");
        std::process::exit(1);
    }
    Arguments {
        target: args[0].to_string(),
        replacement: args[1].to_string(),
        filename: args[2].to_string(),
        output: args[3].to_string()
    }
}


In [6]:
:dep regex = "1.11.1"

In [10]:
use regex::Regex;
use std::fs;

fn replace(target: &str, replacement: &str, text: &str) -> Result<String, regex::Error> {
    let regex = Regex::new(target)?;
    Ok(regex.replace_all(text, replacement).to_string())
}

let args = parse_args();
let content = match fs::read_to_string(&args.filename) {
    Ok(v) => v,
    Err(e) => {
        eprintln!("Failed to read file '{}': {:?}", args.filename, e);
        std::process::exit(1);
    }
};

let replaced = match replace(&args.target, &args.replacement, &content) {
    Ok(v) => v,
    Err(e) => {
        eprintln!("Failed to replace text: {:?}", e);
        std::process::exit(1);
    }
};

match fs::write(&args.output, &replaced) {
    Ok(v) => v,
    Err(e) => {
        eprintln!("Failed to write to file '{}': {:?}", args.filename, e);
        std::process::exit(1);
    }
};

// sample input:
// Rust RUST sample.txt sample-.txt